In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from PIL import Image
from utils.dataloader import YoloDataset, yolo_dataset_collate
from utils.utils import weights_init, load_weights, get_anchors, get_lr
from model.yolo_body import YoloBody
from model.loss import YOLOLoss
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import warnings

In [2]:
dataset_path = './dataset/apples/'
anchors_path = 'model_data/yolo_anchors.txt'
model_path = 'model_data/yolo_weights.pth'
save_path = 'saved_models/'

num_classes = 1
input_shape = [416, 416]
anchors_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]

batch_size = 8
num_workers = 0
lr = 1e-3
n_epoch = 50
cuda = True

Freeze_Train = True

anchors, num_anchors = get_anchors(anchors_path)
with open(dataset_path + 'sets/train.txt', 'r') as file:
    train_splits = file.read().splitlines()
with open(dataset_path + 'sets/val.txt', 'r') as file:
    val_splits = file.read().splitlines()    
print(len(train_splits), len(val_splits))

896 112


In [3]:
model = YoloBody(anchors_mask, num_classes)
weights_init(model)
model = load_weights(model, model_path, device = 'cuda')
model_train = model.train().cuda()

initialize network with normal type
Load weights model_data/yolo_weights.pth.


In [4]:
train_dataset = YoloDataset(dataset_path, train_splits, input_shape, num_classes, train = True)
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = batch_size, num_workers = num_workers, pin_memory=True,
                                    drop_last=True, collate_fn=yolo_dataset_collate)
val_dataset = YoloDataset(dataset_path, val_splits, input_shape, num_classes, train = False)
val_dataloader = DataLoader(val_dataset, shuffle = False, batch_size = 1, num_workers = num_workers, pin_memory=True,
                                    drop_last=False, collate_fn=yolo_dataset_collate)

In [5]:
yolo_loss = YOLOLoss(anchors, num_classes, input_shape, cuda, anchors_mask)
optimizer = optim.Adam(model_train.parameters(), lr, weight_decay = 5e-4)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.94)

if Freeze_Train:
    for param in model.backbone.parameters():
        param.requires_grad = False
epoch_step = len(train_dataset) // batch_size
epoch_step_val = len(val_dataset) // 1

In [6]:
warnings.filterwarnings('ignore')
for epoch in range(n_epoch):
    loss        = 0
    val_loss    = 0    
    model_train.train()
    print('Start Train')
    with tqdm(total = epoch_step, desc = f'Epoch {epoch + 1}/{n_epoch}',postfix = dict, mininterval = 0.3) as pbar:
        for iteration, batch in enumerate(train_dataloader):
            images, targets = batch[0], batch[1] 
            with torch.no_grad():
                if cuda:
                    images = torch.from_numpy(images).type(torch.FloatTensor).cuda()
                    targets = [torch.from_numpy(ann).type(torch.FloatTensor).cuda() for ann in targets]
                else:
                    images = torch.from_numpy(images).type(torch.FloatTensor)
                    targets = [torch.from_numpy(ann).type(torch.FloatTensor) for ann in targets]
            optimizer.zero_grad()
            outputs = model_train(images)
            
            loss_value_all = 0
            num_pos_all = 0
            for layer in range(len(outputs)):
                loss_item, num_pos = yolo_loss(layer, outputs[layer], targets)
                loss_value_all  += loss_item
                num_pos_all     += num_pos
            loss_value = loss_value_all / num_pos_all
            loss_value.backward()
            optimizer.step()
            loss += loss_value.item()
            pbar.set_postfix(**{'loss'  : loss / (iteration + 1), 
                                'lr'    : get_lr(optimizer)})
            pbar.update(1)            
        print('Finish Train')

    model_train.eval()
    print('Start Validation')
    with tqdm(total=epoch_step_val, desc=f'Epoch {epoch + 1}/{n_epoch}',postfix=dict,mininterval=0.3) as pbar:
        for iteration, batch in enumerate(val_dataloader):
            images, targets = batch[0], batch[1]
            with torch.no_grad():
                if cuda:
                    images  = torch.from_numpy(images).type(torch.FloatTensor).cuda()
                    targets = [torch.from_numpy(ann).type(torch.FloatTensor).cuda() for ann in targets]
                else:
                    images  = torch.from_numpy(images).type(torch.FloatTensor)
                    targets = [torch.from_numpy(ann).type(torch.FloatTensor) for ann in targets]
                optimizer.zero_grad()
                outputs = model_train(images)
                loss_value_all = 0
                num_pos_all = 0
                for l in range(len(outputs)):
                    loss_item, num_pos = yolo_loss(l, outputs[l], targets)
                    loss_value_all  += loss_item
                    num_pos_all     += num_pos
                loss_value = loss_value_all / num_pos_all

            val_loss += loss_value.item()
            pbar.set_postfix(**{'val_loss': val_loss / (iteration + 1)})
            pbar.update(1)
    print('Finish Validation')
    print('Epoch:'+ str(epoch+1) + '/' + str(n_epoch))
    print('Total Loss: %.3f || Val Loss: %.3f ' % (loss / epoch_step, val_loss / epoch_step_val))
    torch.save(model.state_dict(), save_path + '/ep%03d-loss%.3f-val_loss%.3f.pth' % (epoch + 1, loss / epoch_step, val_loss / epoch_step_val))        
        
    lr_scheduler.step()

Start Train


Epoch 1/50:   0%|          | 0/112 [00:00<?, ?it/s<class 'dict'>]

Finish Train
Start Validation


Epoch 1/50:   0%|          | 0/112 [00:00<?, ?it/s<class 'dict'>]

Finish Validation
Epoch:1/50
Total Loss: 132.527 || Val Loss: 11.704 
Start Train


...

Finish Validation
Epoch:49/50
Total Loss: 9.185 || Val Loss: 5.813 
Start Train


Epoch 50/50:   0%|          | 0/112 [00:00<?, ?it/s<class 'dict'>]

Finish Train
Start Validation


Epoch 50/50:   0%|          | 0/112 [00:00<?, ?it/s<class 'dict'>]

Finish Validation
Epoch:50/50
Total Loss: 9.175 || Val Loss: 5.764 


In [8]:
%debug

> f:\machine vision\my_yolo\model\loss.py(120)get_target()
    118                 y_true[b, k, j, i, 3] = math.log(batch_target[t, 3] / anchors[best_n][1])
    119                 y_true[b, k, j, i, 4] = 1
--> 120                 y_true[b, k, j, i, c + 5] = 1
    121 
    122                 box_loss_scale[b, k, j, i] = batch_target[t, 2] * batch_target[t, 3] / in_w / in_h

ipdb> y_true.shape
torch.Size([8, 3, 26, 26, 6])
ipdb> q
